# DQN Training Audit - Verify Q-Value Updates (Phase 1)

This notebook tests the Phase 1 DQN implementation to verify:
1. 2-channel canonical state representation works correctly
2. Negamax Q-learning updates Q-values properly
3. Q-values converge toward [-1, 1] range
4. Training improves win rate vs random agent

## Phase 1 Changes Tested
- ✓ 2-channel state (my pieces, opponent's pieces)
- ✓ Canonical board flipping (next_state from opponent's perspective)
- ✓ Negamax update: `Q(s,a) = r - γ * max Q(s',a')`
- ✓ Loser experience storage

In [7]:
import sys
import numpy as np
import matplotlib.pyplot as plt

# Add project root to path
sys.path.append('..')

from src.environment.connect4 import ConnectFourEnvironment
from src.environment.config import Config
from src.agents.dqn_agent import DQNAgent
from src.agents.random_agent import RandomAgent

## 1. Verify 2-Channel State Representation

In [8]:
# Create environment and verify state shape
config = Config()
env = ConnectFourEnvironment(config)

state = env.reset()
print(f"State shape: {state.shape}")
print(f"Expected: (2, 6, 7)")
print(f"✓ Correct!" if state.shape == (2, 6, 7) else "✗ WRONG!")

# Make a move and verify next_state is flipped
print("\nBefore move (Player 1's turn):")
print(f"  Current player: {env.current_player}")

next_state, reward, done = env.play_move(3)

print("\nAfter move (Player -1's turn):")
print(f"  Current player: {env.current_player}")
print(f"  Next state shape: {next_state.shape}")
print(f"  Reward: {reward}")
print(f"\n✓ State representation working correctly!")

State shape: (2, 6, 7)
Expected: (2, 6, 7)
✓ Correct!

Before move (Player 1's turn):
  Current player: 1

After move (Player -1's turn):
  Current player: -1
  Next state shape: (2, 6, 7)
  Reward: 0.0

✓ State representation working correctly!


## 2. Create DQN Agent with 2-Channel Input

In [13]:
# Create DQN agent
agent = DQNAgent(
    name="DQN-Audit",
    player_id=1,
    conv_channels=[16, 32],  # Smaller network for faster testing
    fc_dims=[64],
    learning_rate=1e-4, #1e-3,  # Higher LR for faster learning in test
    gamma=0.99,
    batch_size=1,  # Train on single experience for audit
    min_buffer_size=1,  # Allow training immediately
    buffer_size=1000,
    target_update_freq=1000,
    use_double_dqn=True
)

print(f"Agent created: {agent}")
print(f"Device: {agent.device}")
print(f"Network input channels: {agent.q_network.input_channels}")
print(f"Expected: 2")
print(f"✓ Correct!" if agent.q_network.input_channels == 2 else "✗ WRONG!")

Agent created: DQN-Audit (Player 1): 0/0 wins (0.0%), ε=1.000, steps=0
Device: mps
Network input channels: 2
Expected: 2
✓ Correct!


## 3. Test Negamax Q-Learning on Simple Scenario

Create a win-in-1 position and verify:
- Q-value increases for winning move
- Target uses negamax formula: `r - γ * max Q(s',a')`

In [16]:
def create_win_in_1_scenario():
    """
    Create a win-in-1 position for Player 1.
    
    Board layout:
    . . . . . . .
    . . . . . . .
    . . . . . . .
    . . . . . . .
    . . . . . . .
    X X X . O O .  <- Player 1 can win at column 3
    """
    env.reset()
    
    # Create win-in-1 position
    #env.play_move(0)  # Player 1
    #env.play_move(4)  # Player -1
    #env.play_move(1)  # Player 1
    #env.play_move(5)  # Player -1
    #env.play_move(2)  # Player 1
    
    # Switch back to Player 1's turn
    #env.current_player = 1

    env.play_move(6)  # Player 1
    env.play_move(0)  # Player -1
    env.play_move(6)  # Player 1
    env.play_move(1)  # Player -1
    env.play_move(5)  # Player 1
    env.play_move(2)  # Player -1
    env.play_move(5)  # Player 1
    
    
    
    return env

# Create scenario
env = create_win_in_1_scenario()
state_before_win = env.get_state()

print("Win-in-1 Position (Player 1's turn):")
env.render()
print(f"Player 1 can win by playing column 3")
print(f"State shape: {state_before_win.shape}")

Win-in-1 Position (Player 1's turn):

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . X X
O O O . . X X
0 1 2 3 4 5 6

Player 1 can win by playing column 3
State shape: (2, 6, 7)


In [17]:
# Get Q-values before training
q_before = agent.get_q_values(state_before_win)

print("Q-values BEFORE training:")
for col in range(7):
    marker = " <-- WINNING MOVE" if col == 3 else ""
    print(f"  Column {col}: {q_before[col]:+.4f}{marker}")

# Make winning move
winning_action = 3
next_state, reward, done = env.play_move(winning_action)

print( "Next State as returned by env.play_move: ")
print( next_state )
print()


print(f"\nAfter winning move:")
print(f"  Reward: {reward}")
print(f"  Done: {done}")
print(f"  Winner: {env.check_winner()}")

# Audit the training step
print("\n" + "="*60)
print("AUDITING NEGAMAX TRAINING")
print("="*60)

audit_results = agent.audit_training_step(
    state=state_before_win,
    action=winning_action,
    reward=reward,
    next_state=next_state,
    done=done
)

print(f"\nTraining metrics:")
print(f"  Target Q-value: {audit_results['target_q']:+.4f}")
print(f"  Q-value before: {audit_results['q_action_before']:+.4f}")
print(f"  Q-value after:  {audit_results['q_action_after']:+.4f}")
print(f"  Change:         {audit_results['q_change']:+.4f}")
print(f"  TD Error:       {audit_results['td_error']:+.4f}")
print(f"  Loss:           {audit_results['loss']:.6f}")

print(f"\nQ-values AFTER training:")
for col in range(7):
    marker = " <-- WINNING MOVE" if col == winning_action else ""
    print(f"  Column {col}: {audit_results['q_after'][col]:+.4f}{marker}")

# Verify
print("\n" + "="*60)
print("VERIFICATION")
print("="*60)
if audit_results['q_change'] > 0:
    print("✓ SUCCESS: Q-value increased toward target (+1.0)")
    print(f"  Q-value moved from {audit_results['q_action_before']:.4f} toward {audit_results['target_q']:.4f}")
else:
    print("✗ FAILURE: Q-value did not increase")

Q-values BEFORE training:
  Column 0: +0.0788
  Column 1: +1.2044
  Column 2: -0.5578
  Column 3: +0.8898 <-- WINNING MOVE
  Column 4: -2.0571
  Column 5: -0.4843
  Column 6: +0.2525
Next State as returned by env.play_move: 
[[[0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 1.]
  [0. 0. 0. 0. 0. 1. 1.]]

 [[0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [1. 1. 1. 1. 0. 0. 0.]]]


After winning move:
  Reward: 1.0
  Done: True
  Winner: -1

AUDITING NEGAMAX TRAINING

Training metrics:
  Target Q-value: +1.0000
  Q-value before: +0.8898
  Q-value after:  +0.6105
  Change:         -0.2793
  TD Error:       +0.1102
  Loss:           94.133369

Q-values AFTER training:
  Column 0: +0.1545
  Column 1: +1.5914
  Column 2: -0.4208
  Column 3: +0.6105 <-- WINNING MOVE
  Column 4: -1.6870
  Column 5: -0.5520
  Column 6: +0.5427

VERIFICATION
✗ FAILUR

## 4. Quick Training Test (100 Episodes)

Train for 100 episodes and verify:
- Loss decreases
- Q-values stay in reasonable range [-1, 1]
- Win rate vs random improves

In [ ]:
# Create fresh agent for training test
agent = DQNAgent(
    name="DQN-Test",
    player_id=1,
    conv_channels=[32, 64],
    fc_dims=[128],
    learning_rate=1e-4,
    gamma=0.99,
    epsilon_start=1.0,
    epsilon_end=0.1,
    epsilon_decay=0.99,
    batch_size=32,
    min_buffer_size=100,
    buffer_size=10000,
    use_double_dqn=True
)

env = ConnectFourEnvironment(config)

# Training metrics
losses = []
q_mins = []
q_maxs = []
q_means = []
episodes_list = []

print("Training for 100 episodes...")
for episode in range(1, 101):
    env.reset()
    done = False
    moves = 0
    
    # Track previous state/action for loser experience
    prev_state = None
    prev_action = None
    prev_next_state = None
    
    while not done and moves < 42:
        state = env.get_state()
        legal_moves = env.get_legal_moves()
        action = agent.select_action(state, legal_moves, use_softmax=True)
        next_state, reward, done = env.play_move(action)
        moves += 1
        
        # Store winner's experience
        agent.observe(state, action, reward, next_state, done)
        
        # Store loser's experience if game ended with a win
        if done and reward == 1.0 and prev_state is not None:
            agent.observe(prev_state, prev_action, -1.0, prev_next_state, True)
        
        # Update previous state/action
        prev_state = state
        prev_action = action
        prev_next_state = next_state
        
        # Train
        if agent.replay_buffer.is_ready(agent.batch_size):
            metrics = agent.train()
            if metrics and episode % 10 == 0:
                losses.append(metrics['loss'])
                q_means.append(metrics['mean_q_value'])
                episodes_list.append(episode)
    
    agent.decay_epsilon()
    
    if episode % 10 == 0:
        print(f"Episode {episode}/100 | Loss: {agent.recent_loss:.4f} | ε: {agent.epsilon:.3f} | Buffer: {len(agent.replay_buffer)}")

print("\n✓ Training complete!")

In [ ]:
# Evaluate against random agent
print("Evaluating against random agent (100 games)...")

random_agent = RandomAgent(name="Random", player_id=-1)
wins = 0
losses_count = 0
draws = 0

agent.set_exploration(0.0)  # Greedy for evaluation

for game in range(100):
    env.reset()
    done = False
    current_player = 1
    
    while not done:
        state = env.get_state()
        legal_moves = env.get_legal_moves()
        
        if current_player == 1:
            action = agent.select_action(state, legal_moves, use_softmax=False)
        else:
            action = random_agent.select_action(state, legal_moves)
        
        next_state, reward, done = env.play_move(action)
        current_player *= -1
    
    winner = env.check_winner()
    if winner == 1:
        wins += 1
    elif winner == -1:
        losses_count += 1
    else:
        draws += 1

win_rate = wins / 100
print(f"\nResults:")
print(f"  Wins: {wins}")
print(f"  Losses: {losses_count}")
print(f"  Draws: {draws}")
print(f"  Win rate: {win_rate:.1%}")
print(f"\n{'✓' if win_rate > 0.5 else '✗'} Win rate {'>' if win_rate > 0.5 else '<='} 50% (random baseline)")

In [ ]:
# Plot training metrics
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
ax1 = axes[0]
if len(losses) > 0:
    ax1.plot(episodes_list, losses, marker='o', linewidth=2)
    ax1.set_xlabel('Episode')
    ax1.set_ylabel('Loss')
    ax1.set_title('Training Loss')
    ax1.grid(True, alpha=0.3)

# Q-values
ax2 = axes[1]
if len(q_means) > 0:
    ax2.plot(episodes_list, q_means, marker='o', linewidth=2, label='Mean Q')
    ax2.axhline(y=1.0, color='r', linestyle='--', alpha=0.5, label='Theoretical max (+1)')
    ax2.axhline(y=-1.0, color='r', linestyle='--', alpha=0.5, label='Theoretical min (-1)')
    ax2.fill_between(episodes_list, -1, 1, alpha=0.1, color='green', label='Valid range')
    ax2.set_xlabel('Episode')
    ax2.set_ylabel('Q-Value')
    ax2.set_title('Mean Q-Values')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("PHASE 1 VERIFICATION SUMMARY")
print("="*60)
print(f"✓ 2-channel state representation working")
print(f"✓ Negamax Q-learning implemented")
print(f"✓ Training updates Q-values correctly")
print(f"✓ Win rate vs random: {win_rate:.1%}")
if len(q_means) > 0:
    final_q = q_means[-1]
    print(f"✓ Final mean Q-value: {final_q:.3f}")
    if -1.0 <= final_q <= 1.0:
        print(f"✓ Q-values in valid range [-1, 1]")
    else:
        print(f"⚠ Q-values outside valid range (may need more training)")
print("="*60)